In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



import warnings
warnings.filterwarnings("ignore")


In [2]:
#https://markets.businessinsider.com/news/nvda-stock?p={page}  ->> Nvidia stock news
#https://markets.businessinsider.com/news/amzn-stock?p={page}  ->> Amazon stock news
#https://markets.businessinsider.com/news/tsla-stock?p={page}   ->> Tesla stock news
#https://markets.businessinsider.com/news/googl-stock?p={page}  ->> Google stock news
#https://markets.businessinsider.com/news/adbe-stock?p={page} ->> Adobe stock news
#https://markets.businessinsider.com/news/axp-stock?p={page}  ->> American Express stock news
#https://markets.businessinsider.com/news/meta-stock?p={page}->> Meta stock news
#https://markets.businessinsider.com/news/spot-stock?p={page}->> Spotify stock news
#https://markets.businessinsider.com/news/msft-stock?p={page}     ->> Microsoft stock news
#https://markets.businessinsider.com/news/jpm-stock?p={page}  -->JPMorgan stock news

## Imports

In [3]:
!pip install -q yfinance

from bs4 import BeautifulSoup
import requests
import pandas as pd
import yfinance as yf
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
# Initialize sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python312\\Scripts\\normalizer.exe' -> 'C:\\Python312\\Scripts\\normalizer.exe.deleteme'

[nltk_data] Downloading package vader_lexicon to C:\Users\Ram
[nltk_data]     Chhabra\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Fetcthing data

In [4]:
def fetch_stock_data(ticker):
    print("Fetching stock prices...")
    stock_data = yf.download(ticker, period = "max")
    stock_data = stock_data[['Close','Open','High','Low']]
    return stock_data

def fetch_news(ticker):
    columns = ['datetime','ticker','source', 'headline' ]
    df = pd.DataFrame(columns=columns)
    counter = 0
    ticker = str(ticker).lower()
    print("Fetching news headlines...")
    for page in tqdm(range(1,200)):
        url = f'https://markets.businessinsider.com/news/{ticker}-stock?p={page}'
        response = requests.get(url)
        html = response.text
        soup = BeautifulSoup (html, 'lxml')
        articles = soup.find_all('div', class_ = 'latest-news__story')
        for article in articles:
            datetime = article.find('time', class_ = 'latest-news__date').get('datetime')
            title = article.find('a', class_ = 'news-link').text
            source = article.find('span', class_ = 'latest-news__source').text
        #   link = article.find('a', class_ = 'news-link').get('href')
            ticker = 'ADBE'       # 'AMZN', 'TSLA', 'GOOGL', 'ADBE', 'AXP', 'META', 'NVDA' ,'SPOT' ,'MSFT' ,'JPM'
            df = pd.concat([pd.DataFrame([[datetime,ticker, source,title]], columns=df.columns), df], ignore_index=True)
            counter += 1
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['date'] = df['datetime'].dt.date
    df['time'] = df['datetime'].dt.time
    df.drop(columns=['datetime'], inplace=True)
    print (f'{counter} headlines scraped from {page+1} pages')
    return df

# Sentiment score Analysis

In [5]:
def calculate_sentiment_scores(headlines_df):
    headlines_df['sentiment'] = headlines_df['headline'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
    headlines_df['date'] = pd.to_datetime(headlines_df['date'])
    
    return headlines_df

def aggregate_sentiment_scores(headlines_df):
    sentiment_summary = headlines_df.groupby('date')['sentiment'].mean()
    return sentiment_summary

def generate_trading_signals(sentiment_summary):
    signals = sentiment_summary.apply(lambda x: 1 if x > 0.2 else (-1 if x < -0.2 else 0))
    return signals

# Simulating Trades

In [6]:
def simulate_trades(stock_data, trading_signals,capital):
    portfolio = []
    position = 0
    buy_price = 0
    quantity =0
    for date, price in (stock_data['Close'].items()):
        if date in trading_signals.index:
            signal = trading_signals.loc[date]
            if signal == 1 and position == 0:  # Buy signal
                position = 1
                buy_price = price
                quantity = capital/price
                capital = capital%price
                portfolio.append({"date": date, "type": "buy", "price": buy_price, "capital":capital})
            elif signal == -1 and position == 1 and (price>buy_price) :  # Sell signal
                position = 0
                sell_price = price
                profit = (sell_price - buy_price)*quantity
                capital = capital+ quantity*sell_price
                portfolio.append({"date": date, "type": "sell", "price": sell_price,"capital":capital, "profit": profit})

    return pd.DataFrame(portfolio)
 

# calculate portfolio metrics

In [7]:
def calculate_portfolio_metrics(portfolio):
    total_trades = len(portfolio) // 2
    wins = portfolio[portfolio['type'] == 'sell']['profit'] > 0
    win_percentage = wins.mean() * 100
    total_profit = portfolio[portfolio['type'] == 'sell']['profit'].sum()
    
    return total_trades, win_percentage, total_profit

def calculate_sharpe_ratio(portfolio, risk_free_rate=0.01):
    daily_returns = portfolio[portfolio['type'] == 'sell']['profit']
    excess_returns = daily_returns - risk_free_rate
    sharpe_ratio = excess_returns.mean() / excess_returns.std()
    return sharpe_ratio

def calculate_max_drawdown(portfolio):
    portfolio['cumulative_profit'] = portfolio['profit'].cumsum()
    cumulative_max = portfolio['cumulative_profit'].cummax()
    drawdown = portfolio['cumulative_profit'] - cumulative_max
    max_drawdown = drawdown.min()
    return max_drawdown

# Plot and calculations

In [8]:
import plotly.graph_objects as go

def plot_signals(stock_data, portfolio, ticker):
    start_date = portfolio['date'].min()
    end_date = portfolio['date'].max()

    # Filter stock data to the date range available in the portfolio
    stock_data = stock_data[(stock_data.index >= start_date) & (stock_data.index <= end_date)]
    
    buy_signals = portfolio[portfolio['type'] == 'buy']
    sell_signals = portfolio[portfolio['type'] == 'sell']
    fig = go.Figure()
    # Add stock price trace
    fig.add_trace(go.Scatter(
        x=stock_data.index, 
        y=stock_data['Close'],
        mode='lines',
        name='Stock Price',
        line=dict(color='blue')
    ))

    # Add buy signals
    fig.add_trace(go.Scatter(
        x=buy_signals['date'], 
        y=buy_signals['price'],
        mode='markers',
        name='Buy Signal',
        marker=dict(symbol='triangle-up', color='green', size=10)
    ))

    # Add sell signals
    fig.add_trace(go.Scatter(
        x=sell_signals['date'], 
        y=sell_signals['price'],
        mode='markers',
        name='Sell Signal',
        marker=dict(symbol='triangle-down', color='red', size=10)
    ))
    # Update layout for better presentation
    fig.update_layout(
        title=f'Stock Price with Buy and Sell Signals for {ticker}',
        xaxis_title='Date',
        yaxis_title='Price',
        legend_title='Legend',
        hovermode='x'
    )
    fig.show()

# Main function 

In [9]:
def final(ticker, rate, intial, df):

    # Fetch stock data
    stock_data = fetch_stock_data(ticker)
    

    # Fetch news data
    news_data = df
    
    # Calculate sentiment scores
    news_data_with_scores = calculate_sentiment_scores(news_data)
    # Aggregate sentiment scores by date
    sentiment_summary = aggregate_sentiment_scores(news_data_with_scores)
    # Generate trading signals
    trading_signals = generate_trading_signals(sentiment_summary)
    # Simulate trades
    print("\nSimulating trades...")
    portfolio = simulate_trades(stock_data, trading_signals, intial)
    # Calculate portfolio metrics
    total_trades, win_percentage, total_profit = calculate_portfolio_metrics(portfolio)
    # Print the portfolio
    print(portfolio)
    print(f"\nInitial capital: ${intial}")
    print(f"Total Trades: {total_trades}")
    print(f"Win Percentage: {win_percentage:.2f}%")
    print(f"Total Portfolio Returns: ${total_profit:.2f}")
    print(f"Sharpe ratio:{calculate_sharpe_ratio(portfolio, rate):.2f} with risk free rate of {rate}")
    print(f"Max drawdown: {calculate_max_drawdown(portfolio)}")
    # Plot buy and sell signals
    plot_signals(stock_data, portfolio, ticker)

In [10]:
adobe_news = fetch_news("ADBE")

Fetching news headlines...


  0%|          | 0/199 [00:00<?, ?it/s]

1700 headlines scraped from 200 pages


In [11]:
final("ADBE", 0.01, 1000, adobe_news)

Fetching stock prices...


[*********************100%%**********************]  1 of 1 completed



Simulating trades...
         date  type       price       capital       profit
0  2013-06-18   buy   43.360001      2.719986          NaN
1  2014-09-16  sell   70.730003   1633.946978   631.226992
2  2015-01-14   buy   69.989998     24.177027          NaN
3  2016-10-20  sell  109.029999   2569.529826   911.405822
4  2016-10-27   buy  107.070000    106.919833          NaN
5  2017-07-25  sell  148.210007   3663.752128   987.302469
6  2017-07-27   buy  146.199997      8.752205          NaN
7  2017-08-14  sell  149.160004   3746.681701    74.177367
8  2017-09-07   buy  155.470001     15.401671          NaN
9  2018-12-14  sell  230.000000   5558.186674  1796.103302
10 2019-01-15   buy  242.360001    226.266661          NaN
11 2019-03-26  sell  265.589996   6317.200704   532.747369
12 2019-06-06   buy  274.799988    271.600972          NaN
13 2019-06-25  sell  287.970001   6891.558670   302.756993
14 2019-06-26   buy  288.720001    250.998642          NaN
15 2019-07-03  sell  305.700012   

In [12]:
amazon_news = fetch_news("AMZN")

Fetching news headlines...


  0%|          | 0/199 [00:00<?, ?it/s]

1700 headlines scraped from 200 pages


In [13]:
final("AMZN", 0.01, 1000, amazon_news)

Fetching stock prices...


[*********************100%%**********************]  1 of 1 completed



Simulating trades...
         date  type       price       capital       profit
0  2013-06-18   buy   14.088000     13.839979          NaN
1  2014-09-16  sell   16.388000   1177.099501   163.259522
2  2015-01-14   buy   14.663500      4.019514          NaN
3  2016-10-20  sell   40.515999   3256.405545  2075.286530
4  2016-10-27   buy   40.917999     23.883603          NaN
5  2017-07-25  sell   51.993500   4161.718393   881.429245
6  2017-07-27   buy   52.299999     30.018453          NaN
7  2018-12-14  sell   79.595497   6363.747884  2172.011038
8  2019-01-15   buy   83.727997      0.420125          NaN
9  2019-03-26  sell   89.188004   6779.155916   414.987907
10 2019-06-06   buy   87.718002     24.869737          NaN
11 2019-06-25  sell   93.913498   7282.835356   478.809704
12 2019-06-26   buy   94.891502     71.081175          NaN
13 2019-07-03  sell   96.949997   7511.904092   157.987561
14 2019-07-08   buy   97.615997     93.088296          NaN
15 2019-07-11  sell  100.053497   